In [ ]:
from nbdev import *

In [ ]:
#hide
from hecss.core import *

# VASP Tutorial

This example uses VASP to calculate forces and energies. This is a non-free calculator which needs to be installed/setup separately. This example can be easily adapted to some other DFT calculator (e.g. QuantumEspresso, AbInit etc.)

In [ ]:
#vasp

# Import VASP calculator and unit modules
from ase.calculators.vasp import Vasp2
from ase import units as un
from os.path import isfile
import os

# The sample generator, monitoring display and dfset writer
from hecss.core import HECSS, write_dfset

# clear_output function for nice monitoring display
from IPython.display import clear_output

In [ ]:
#vasp

# Quick test using conventional unit cell
supercell = '1x1x1'

### More realistic test

You can run more realistic test using 2x2x2 (or even larger) supercell. The results generated by such calculation are included in the `example/VASP_3C-SiC_calculated` directory. You can run post-processing 
on these results by copying the `2x2x2` subdirectory to `example/VASP_3C-SiC` and changing the value 
of the `supercell` variable:
```python
supercell = '2x2x2'
```
If you want to run your own calulation be prepared for some substantial computational resources required for such a job. Better test first with single unit cell.

In [ ]:
#vasp

# Directory in which our project resides
base_dir = f'example/VASP_3C-SiC/{supercell}/'

In [ ]:
#vasp

# Desired number of samples.
# Here 4 is minimal in 3C-SiC 
# to get somewhat decent phonons 
# with 10 Bohr cutoff.
N = 4

In [ ]:
#vasp

# Temperature (K)
T = 300

In [ ]:
#vasp

# Read the structure (previously calculated unit(super) cell)
# The command argument is specific to the cluster setup
calc = Vasp2(label='cryst', directory=f'{base_dir}/sc_{supercell}/', restart=True)

# This just makes a copy of atoms object
# Do not generate supercell here - your atom ordering will be wrong!
cryst = calc.atoms.repeat(1)

If you have magmoms in your system you need to use following 
temporary fix for a bug in magmom handling in Vasp2 calculator:
```python
if 'magmom' in calc.list_float_params:
    calc.list_float_params['magmom'] = cryst.get_initial_magnetic_moments()
```
Just copy the above code to a new cell here and execute it.

### Calculator setup

You need to configure your own environment for VASP. The `run-vasp` script located in the root directory is an example from my own system. You need to modify it to fit your system.
Running `run-vasp` in a given directory should submit a VASP job to your cluster *and* wait for it to end. Example of such job script from my system (small cluster with SLURM job menager) is included in `example/scripts` directory.

In [ ]:
#vasp

# Setup the calculator - single point energy calculation
# The details will change here from case to case
# We are using run-vasp from the current directory!
calc.set(directory=f'{base_dir}/calc')
calc.set(command=f'{os.getcwd()}/run-vasp -J "hecss-3C-SiC-{supercell}"')
calc.set(nsw=0)
cryst.set_calculator(calc)

You should probably check the calculator setup and the stress tensor of the supercell to make sure it is in equilibrium before running long sequence of
DFT calculations. Here is an example:

In [ ]:
#vasp

print('Stress tensor: ', end='')
for ss in calc.get_stress()/un.GPa:
    print(f'{ss:.3f}', end=' ')
print('GPa')

Stress tensor: 0.017 0.017 0.017 0.000 0.000 0.000 GPa


In [ ]:
#vasp

# Space for results
confs = []
dfsetfn = f'{base_dir}/phon/DFSET_T{T:.1f}K'
calc_dir = f'{base_dir}/calc/T{T:.1f}K/'

In [ ]:
#vasp

# Build the sampler
sampler = HECSS(cryst, calc, T, directory=calc_dir,
                # sigma=3,  # Use if the energy distribution comes out too narrow
                # reuse_base=f'{base_dir}/calc/' # Use if you want to reuse the base calc
               )

In [ ]:
#vasp

# Iterate over samples (conf == n, i, x, f, e) collect the configurations
# and write displacement-force data to the dfsetfn file
# You can continue the iteration by manually increasing N and just
# re-running this loop. It will continue from the last computed sample.
for conf in sampler:
    # Collect results
    confs.append(conf)    
    write_dfset(dfsetfn, conf)
    
    clear_output(wait=True)
    
    # Check if we have enough samples
    if len(confs) >= N:
        break
    
    # Check for the manual stop file in calc_dir
    if isfile(f'{calc_dir}/STOP_HECSS'):
        os.remove(f'{calc_dir}/STOP_HECSS')
        break

Sample:0100  a: 35.0%  w:1.1136  <w>:1.0667  alpha: 5.275e-01                


In [ ]:
#vasp

# Need more samples. Increase N and run the loop above again.
N = 100